In [1]:
# import necessary modules
import cv2 as cv
import numpy as np
import glob


In [2]:
# termination criteria
criteria = (cv.TermCriteria_EPS + cv.TermCriteria_MAX_ITER, 30, 0.001)

In [3]:
# prepare object points
objp = np.zeros((8*11,3), np.float32) # zero array for 8 x 11 circle board
objp[:,:2] = np.mgrid[0:11,0:8].T.reshape(-1,2)  # format shape of array

# arrays to store object points and image points
objpoints = []
imgpoints = []

In [4]:
# read the image file(s)
images = glob.glob("*.JPG")

#img = cv.imread("DSC_2935.JPG")

# resize the image to make it more manageable
#reimg = cv.resize(img, (1200,800))
#cv.imwrite("reimg.png", reimg)


In [11]:
counter, failed, success = 1, 0, 0
for fname in images:
    percent_done = counter * 100 / len(images)
    print("{0:.1f}% complete".format(percent_done))
    counter += 1
    img = cv.imread(fname)
    # resize the image to make it more manageable
    reimg = cv.resize(img, (3000,2000))

    # convert to gray
    gray = cv.cvtColor(reimg, cv.COLOR_BGR2GRAY)
    
    shape = (8,11)
    
    # find circles on image
    ret, centres = cv.findCirclesGrid(gray, shape, flags=
                                      cv.CALIB_CB_ASYMMETRIC_GRID 
                                      + cv.CALIB_CB_CLUSTERING)

    if ret == True: 
        success += 1
        print("Circle pattern is found...")
        objpoints.append(objp)
    
        centres2 = centres
        #centres2 = cv.cornerSubPix(gray, centres, (11,11), (-1,-1), criteria)


        imgpoints.append(centres2)

        # draw and display the patterns
        img = cv.drawChessboardCorners(reimg, shape, centres2, ret)

        cv.imshow("img", reimg)

        cv.waitKey(500)
        cv.destroyAllWindows()
        
    
    else:
        failed += 1
        print("Circle pattern not found...")
        

    
print("Succeeded for {}/{} images".format(success,len(images)))
    

10.0% complete
Circle pattern not found...
20.0% complete
Circle pattern not found...
30.0% complete
Circle pattern not found...
40.0% complete
Circle pattern not found...
50.0% complete
Circle pattern not found...
60.0% complete
Circle pattern not found...
70.0% complete
Circle pattern not found...
80.0% complete
Circle pattern not found...
90.0% complete
Circle pattern not found...
100.0% complete
Circle pattern not found...
Succeeded for 0/10 images


In [12]:
# camera calibration
h, w = img.shape[:2]

rms, camera_matrix, dist_coefs, rvecs, tvecs = cv.calibrateCamera(
                                                objpoints, imgpoints, 
                                                (w,h), None, None)


print("\nRMS", rms)
print("Camera Matrix: \n", camera_matrix)
print("distortion coefficients: ", dist_coefs.ravel())


error: OpenCV(3.4.3) D:\Build\OpenCV\opencv-3.4.3\modules\calib3d\src\calibration.cpp:3146: error: (-215:Assertion failed) ni == ni1 in function 'cv::collectCalibrationData'


In [ ]:
newcameramtx, roi = cv.getOptimalNewCameraMatrix(camera_matrix, 
                                                dist_coefs, (w,h), 1, (w,h))

print("New Camera Matrix: \n", newcameramtx)
print("ROI = ", roi)


In [ ]:
# undistort the image

dst = cv.undistort(img, camera_matrix, dist_coefs, None, newcameramtx)

In [ ]:
# crop and display image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]



In [ ]:
# show before and after distortion
cv.imshow("The result!", dst)
cv.imshow("Original", img)
cv.waitKey(0)

cv.destroyAllWindows()